# Projet Datascientest - Compagnon immo

<code>mar25_bds_compagnon_immo_1</code>

---

## Pré-processing de données - v1.0

---

In [8]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('future.no_silent_downcasting', True)

---

### **Preprocessing du dataset sur les ventes immobilières**

---

In [5]:
url_sales = 'https://raw.githubusercontent.com/klopstock-dviz/immo_vis/master/data/ech_annonces_ventes_68.csv'

sales_df = pd.read_csv(url_sales, sep=';', index_col="idannonce")

display(sales_df.head())

,type_annonceur,typedebien,typedetransaction,etage,surface,surface_terrain,nb_pieces,prix_bien,prix_maison,prix_terrain,mensualiteFinance,balcon,eau,bain,dpeL,dpeC,mapCoordonneesLatitude,mapCoordonneesLongitude,annonce_exclusive,nb_etages,parking,places_parking,cave,exposition,ges_class,annee_construction,nb_toilettes,nb_terraces,videophone,porte_digicode,surface_balcon,ascenseur,nb_logements_copro,charges_copro,chauffage_energie,chauffage_systeme,chauffage_mode,categorie_annonceur,logement_neuf,duree_int,typedebien_lite,date,INSEE_COM,IRIS,CODE_IRIS,TYP_IRIS_x,TYP_IRIS_y,GRD_QUART,UU2010,REG,DEP,loyer_m2_median_n6,nb_log_n6,taux_rendement_n6,loyer_m2_median_n7,nb_log_n7,taux_rendement_n7,prix_m2_vente
idannonce,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
hektor-robischung-2862,pr,a,v,0,64,NaN,3,128400,NaN,NaN,0,1,0,1,C,118.0,47.55424,7.34766,Non,1.0,NaN,NaN,False,0,C,2010.0,1.0,NaN,NaN,NaN,NaN,False,21.0,NaN,Gaz,NaN,Individuel,a,n,NaN,a,2023-04,68325,0,683250000,Z,Z,6832500,68113,44,68,NaN,NaN,NaN,NaN,NaN,NaN,2006.25
apimo-82452515,pr,m,v,0,85,4950.0,4,189000,NaN,NaN,0,0,0,0,0,NaN,48.17073,7.10498,0,NaN,NaN,NaN,NaN,Sud,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Électrique,Radiateur,Individuel,a,n,NaN,m,2023-04,68044,0,680440000,Z,Z,6804400,68000,44,68,NaN,NaN,NaN,NaN,NaN,NaN,2223.53
immo-facile-52471952,pr,m,v,0,187,3182.0,9,282900,NaN,NaN,0,0,0,0,E,251.0,48.16989,7.13037,Oui,NaN,NaN,NaN,NaN,0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,NaN,m,2023-04,68044,0,680440000,Z,Z,6804400,68000,44,68,NaN,NaN,NaN,NaN,NaN,NaN,1512.83
immo-facile-52477215,pr,a,v,0,76,NaN,4,79000,NaN,NaN,0,0,1,0,G,379.0,48.17061,7.11870,Oui,1.0,NaN,NaN,True,0,G,NaN,1.0,1.0,NaN,NaN,NaN,False,2.0,180.0,Fioul,Radiateur,Collectif,a,n,8.0,a,2023-04,68044,0,680440000,Z,Z,6804400,68000,44,68,7.93,3.0,0.092,7.93,3.0,0.092,1039.47
ag681088-383601401,pr,m,v,0,90,3071.0,5,317000,NaN,NaN,0,0,0,1,NS,NaN,47.88536,7.19668,Non,NaN,NaN,2.0,True,0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,NaN,m,2023-04,68159,0,681590000,Z,Z,6815900,68000,44,68,9.77,2.0,0.033,9.77,2.0,0.033,3522.22


### 1) Suppression colonnes

In [1001]:
sales_df = sales_df.drop(['type_annonceur',
                          'parking', 
                          'places_parking', 
                          'surface_balcon',
                          'charges_copro',
                          'duree_int',
                          'loyer_m2_median_n6', 
                          'nb_log_n6',
                          'taux_rendement_n6',
                          'loyer_m2_median_n7',
                          'nb_log_n7',
                          'taux_rendement_n7'], axis=1)

### 2) Gestion des manquants

In [1004]:
# % de manquants avant traitement
round((sales_df.isna().sum().sum() / sales_df.size) * 100, 2)

24.41

In [1006]:
sales_df.loc[(sales_df['surface_terrain'].isna()) & (sales_df['prix_terrain'].isna()), 'surface_terrain'] = 0

sales_df.loc[(sales_df['prix_maison'].isna()) & (sales_df['typedebien'].isin(['a', 'an']).isna()), 'prix_maison'] = 0

sales_df.loc[(sales_df['prix_terrain'].isna()) & (sales_df['surface_terrain'].isna()), 'prix_terrain'] = 0

sales_df.loc[(sales_df['dpeC'].isna()) & (sales_df['dpeL'].isin(['0', 'NS', 'VI'])), 'dpeC'] = 0
sales_df['dpeC'] = sales_df['dpeC'].fillna(0)

sales_df.loc[(sales_df['nb_etages'].isna()) & (sales_df['etage'] == 0), 'nb_etages'] = 0
sales_df['nb_etages'] = sales_df['nb_etages'].fillna(0)

sales_df['cave'] = sales_df['cave'].fillna("False")

sales_df['ges_class'] = sales_df['ges_class'].fillna('NS')

sales_df['annee_construction'] = sales_df['annee_construction'].fillna(
    sales_df.groupby('GRD_QUART')['annee_construction'].transform('median')
)

sales_df['nb_toilettes'] = sales_df['nb_toilettes'].fillna(1)

sales_df['nb_terraces'] = sales_df['nb_terraces'].fillna(0)

sales_df['videophone'] = sales_df['videophone'].fillna(0)

sales_df['porte_digicode'] = sales_df['porte_digicode'].fillna(0)

sales_df['ascenseur'] = np.where(sales_df['ascenseur'].isna() & (sales_df['nb_etages'] > 7), 'True', 'False')

sales_df['nb_logements_copro'] = sales_df['nb_logements_copro'].fillna(0)

sales_df.loc[(sales_df['chauffage_energie'].isna()) & 
             (sales_df['chauffage_systeme'].isin(['Climatisation révérsible', 'Convecteur', 'Radiateur'])), 'chauffage_energie'] = 'Électrique'

sales_df['chauffage_energie'] = sales_df['chauffage_energie'].fillna('Non spécifié')

sales_df.loc[(sales_df['chauffage_systeme'].isna()) & 
             (sales_df['chauffage_energie'] == 'Bois'), 'chauffage_systeme'] = 'Poêle à Bois'
sales_df.loc[(sales_df['chauffage_systeme'].isna()) & 
             (sales_df['chauffage_energie'] == 'Gaz'), 'chauffage_systeme'] = 'Sol'
sales_df['chauffage_systeme'] = sales_df['chauffage_systeme'].fillna('Autre')

sales_df['chauffage_mode'] = sales_df['chauffage_mode'].fillna('Non spécifié')

sales_df.loc[(sales_df['logement_neuf'].isna()) & (sales_df['typedebien'].isin(['an', 'mn'])), 'logement_neuf'] = 1
sales_df.loc[(sales_df['logement_neuf'].isna()) & (sales_df['typedebien'].isin(['a', 'm', 'l'])), 'logement_neuf'] = 0
sales_df['logement_neuf'] = sales_df['logement_neuf'].fillna(0)

sales_df.loc[(sales_df['prix_terrain'].isna()) & (sales_df['surface_terrain'].isna()), 'prix_terrain'] = 0

sales_df['prix_terrain'] = sales_df['prix_terrain'].fillna(
    sales_df.groupby('GRD_QUART')['prix_terrain'].transform('median')
)

sales_df['prix_maison'] = sales_df['prix_maison'].fillna(
    sales_df.groupby('GRD_QUART')['prix_maison'].transform('median')
)

chauffage_energie_mapping = {'Gaz': 'G', 'Non spécifié': 'NS', 'Électrique': 'E', 'Fioul': 'F', 'Bois': 'B',
       'Gaz, Électrique': 'GE', 'Bois, Fioul': 'BF', 'Gaz, Bois': 'GB', 'Électrique, Bois': 'EB',
       'Fioul, Bois': 'FB', 'Bois, Électrique': 'BE'}

# Evite les problèmes d'encodage
sales_df['chauffage_energie'] = sales_df['chauffage_energie'].replace(chauffage_energie_mapping)

# On se base sur le chauffage mode
sales_df = sales_df.drop('chauffage_systeme', axis=1)

In [1008]:
sales_df = sales_df.dropna(subset=['surface_terrain', 'annee_construction', 'prix_terrain'], axis=0)

In [1010]:
# % de manquants après traitement
round((sales_df.isna().sum().sum() / sales_df.size) * 100, 2)

0.0

### 3) Gestion des types

In [1012]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17539 entries, hektor-robischung-2862 to 135755855
Data columns (total 45 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   typedebien               17539 non-null  object 
 1   typedetransaction        17539 non-null  object 
 2   etage                    17539 non-null  int64  
 3   surface                  17539 non-null  int64  
 4   surface_terrain          17539 non-null  float64
 5   nb_pieces                17539 non-null  int64  
 6   prix_bien                17539 non-null  int64  
 7   prix_maison              17539 non-null  float64
 8   prix_terrain             17539 non-null  float64
 9   mensualiteFinance        17539 non-null  int64  
 10  balcon                   17539 non-null  int64  
 11  eau                      17539 non-null  int64  
 12  bain                     17539 non-null  int64  
 13  dpeL                     17539 non-null  object 
 14  dp

In [1014]:
sales_df['nb_etages'] = sales_df['nb_etages'].astype(int)
sales_df['nb_toilettes'] = sales_df['nb_toilettes'].astype(int)
sales_df['nb_terraces'] = sales_df['nb_terraces'].astype(int)

sales_df['videophone'] = np.where(sales_df['videophone'] == True,  1, 0).astype(int)
sales_df['porte_digicode'] = np.where(sales_df['porte_digicode'] == True,  1, 0).astype(int)
sales_df['cave'] = np.where(sales_df['cave'] == True,  1, 0).astype(int)
sales_df['ascenseur'] = np.where(sales_df['cave'] == True,  1, 0).astype(int)

sales_df['logement_neuf'] = sales_df['logement_neuf'].replace({'n': 0, 'o': 1}).astype(int)

sales_df['annonce_exclusive'] = sales_df['annonce_exclusive'].replace({'Non': 0, 'Oui': 1, 'O': 1}).astype(int)
sales_df['date'] = pd.to_datetime(sales_df['date'], format='%Y-%m')

# Création de colonnes pour les dates
sales_df['mois'] = sales_df['date'].dt.month
sales_df['annee'] = sales_df['date'].dt.year
 
sales_df.dtypes

typedebien                         object
typedetransaction                  object
etage                               int64
surface                             int64
surface_terrain                   float64
nb_pieces                           int64
prix_bien                           int64
prix_maison                       float64
prix_terrain                      float64
mensualiteFinance                   int64
balcon                              int64
eau                                 int64
bain                                int64
dpeL                               object
dpeC                              float64
mapCoordonneesLatitude            float64
mapCoordonneesLongitude           float64
annonce_exclusive                   int32
nb_etages                           int32
cave                                int32
exposition                         object
ges_class                          object
annee_construction                float64
nb_toilettes                      

### 4) Encodage

In [1016]:
print("Nombres de variables catégorielles :", len(sales_df.select_dtypes('object').columns))

Nombres de variables catégorielles : 11


In [1018]:
sales_df = pd.get_dummies(sales_df, columns=['categorie_annonceur', 'typedebien_lite', 'TYP_IRIS_x', 'TYP_IRIS_y', 
                                             'typedebien', 'typedetransaction', 'exposition', 'chauffage_energie'], 
                          drop_first=True)

# Encodage Ordinal
ges_class_mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'NS': 7, 'VI': 8}
dpeL_mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, '0': 7, 'NS': 8, 'VI': 9}
chauffage_mode_mapping = {'Non spécifié': 0, 'Individuel': 1, 'Collectif': 2, 'Central': 3,
                          'Individuel, Central': 4, 'Collectif, Central': 5, 'Collectif, Individuel': 6}

sales_df['ges_class'] = sales_df['ges_class'].replace(ges_class_mapping).astype(int)
sales_df['dpeL'] = sales_df['dpeL'].replace(dpeL_mapping).astype(int)
sales_df['chauffage_mode'] = sales_df['chauffage_mode'].replace(chauffage_mode_mapping).astype(int)

In [1020]:
print("Variables catégorielles restantes :", len(sales_df.select_dtypes('object').columns))

Variables catégorielles restantes : 0


### 5) Dump

In [1026]:
output_path = '../data/processed/sales_data_transformed.csv'
sales_df.to_csv(output_path, index=False)

---

### **Preprocessing du dataset sur les locations immobilières**

---

In [198]:
url_rentals = 'https://raw.githubusercontent.com/klopstock-dviz/immo_vis/master/data/ech_annonces_locations_68.csv'

rentals_df = pd.read_csv(url_rentals, sep=';', index_col="idannonce")

display(rentals_df.head())

,type_annonceur,typedebien,typedetransaction,etage,surface,surface_terrain,nb_pieces,prix_bien,prix_maison,prix_terrain,mensualiteFinance,balcon,eau,bain,dpeL,dpeC,mapCoordonneesLatitude,mapCoordonneesLongitude,annonce_exclusive,nb_etages,parking,places_parking,cave,exposition,ges_class,annee_construction,nb_toilettes,nb_terraces,videophone,porte_digicode,surface_balcon,ascenseur,nb_logements_copro,charges_copro,chauffage_energie,chauffage_systeme,chauffage_mode,categorie_annonceur,logement_neuf,duree_int,typedebien_lite,date,INSEE_COM,IRIS,CODE_IRIS,TYP_IRIS_x,TYP_IRIS_y,GRD_QUART,UU2010,REG,DEP
idannonce,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
immo-facile-52417007,pr,m,l,0,116,NaN,5,1250,NaN,NaN,0,0,0,1,D,226.0,47.88299,7.19650,0,1.0,NaN,NaN,True,0,B,1987.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Électrique,Radiateur,Individuel,a,n,15.0,m,2023-04,68159,0,681590000,Z,Z,6815900,68000,44,68
ag672097-382986683,pr,a,l,3,44,NaN,3,600,NaN,NaN,0,0,1,0,D,220.0,47.75081,7.32245,0,3.0,NaN,NaN,NaN,0,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Électrique,NaN,Individuel,a,n,NaN,a,2023-04,68224,1201,682241201,H,H,6822401,68701,44,68
ag672097-383464082,pr,a,l,1,53,NaN,2,498,NaN,NaN,0,1,1,0,E,272.0,47.75094,7.32239,0,NaN,NaN,NaN,NaN,0,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gaz,NaN,Individuel,a,n,NaN,a,2023-04,68224,1201,682241201,H,H,6822401,68701,44,68
ag680864-383044166,pr,a,l,0,41,NaN,2,460,NaN,NaN,0,0,1,1,VI,NaN,47.75204,7.32135,0,4.0,NaN,NaN,False,0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,False,NaN,NaN,Gaz,Radiateur,Collectif,a,n,NaN,a,2023-04,68224,1201,682241201,H,H,6822401,68701,44,68
ag672097-380998252,pr,a,l,2,74,NaN,4,695,NaN,NaN,0,0,0,1,D,206.0,47.75075,7.32253,0,3.0,NaN,NaN,NaN,0,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gaz,NaN,Individuel,a,n,20.0,a,2023-04,68224,1201,682241201,H,H,6822401,68701,44,68


### 1) Suppression colonnes

In [1106]:
rentals_df = rentals_df.drop(['prix_maison', 
                              'prix_terrain',
                              'surface_terrain', 
                              'parking', 
                              'places_parking', 
                              'charges_copro', 
                              'surface_balcon', 
                              'nb_logements_copro',
                              'duree_int'], axis=1)

### 2) Gestion des manquants

In [1072]:
# % de manquants avant traitement
round((rentals_df.isna().sum().sum() / rentals_df.size) * 100, 2)

17.93

In [1108]:
rentals_df.loc[(rentals_df['dpeC'].isna()) & (rentals_df['dpeL'].isin(['0', 'NS', 'VI'])), 'dpeC'] = 0
rentals_df['dpeC'] = rentals_df['dpeC'].fillna(0)

rentals_df.loc[(rentals_df['nb_etages'].isna()) & (rentals_df['etage'] == 0), 'nb_etages'] = 0
rentals_df['nb_etages'] = rentals_df['nb_etages'].fillna(0)

rentals_df['cave'] = rentals_df['cave'].fillna("False")

rentals_df['ges_class'] = rentals_df['ges_class'].fillna('NS')

rentals_df['annee_construction'] = rentals_df['annee_construction'].fillna(
    rentals_df.groupby('GRD_QUART')['annee_construction'].transform('median')
)

rentals_df['nb_toilettes'] = rentals_df['nb_toilettes'].fillna(1)

rentals_df['nb_terraces'] = rentals_df['nb_terraces'].fillna(0)

rentals_df['videophone'] = rentals_df['videophone'].fillna(0)

rentals_df['porte_digicode'] = rentals_df['porte_digicode'].fillna(0)

rentals_df['ascenseur'] = np.where(rentals_df['ascenseur'].isna() & (rentals_df['nb_etages'] > 7), 'True', 'False')

rentals_df.loc[(rentals_df['chauffage_energie'].isna()) & 
             (rentals_df['chauffage_systeme'].isin(['Climatisation révérsible', 'Convecteur', 'Radiateur'])), 'chauffage_energie'] = 'Électrique'

rentals_df['chauffage_energie'] = rentals_df['chauffage_energie'].fillna('Non spécifié')

rentals_df.loc[(rentals_df['chauffage_systeme'].isna()) & 
             (rentals_df['chauffage_energie'] == 'Bois'), 'chauffage_systeme'] = 'Poêle à Bois'
rentals_df.loc[(rentals_df['chauffage_systeme'].isna()) & 
             (rentals_df['chauffage_energie'] == 'Gaz'), 'chauffage_systeme'] = 'Sol'
rentals_df['chauffage_systeme'] = rentals_df['chauffage_systeme'].fillna('Autre')

rentals_df['chauffage_mode'] = rentals_df['chauffage_mode'].fillna('Non spécifié')

rentals_df.loc[(rentals_df['logement_neuf'].isna()) & (rentals_df['typedebien'].isin(['an', 'mn'])), 'logement_neuf'] = 1
rentals_df.loc[(rentals_df['logement_neuf'].isna()) & (rentals_df['typedebien'].isin(['a', 'm', 'l'])), 'logement_neuf'] = 0
rentals_df['logement_neuf'] = rentals_df['logement_neuf'].fillna(0)

chauffage_energie_mapping = {'Gaz': 'G', 'Non spécifié': 'NS', 'Électrique': 'E', 'Fioul': 'F', 'Bois': 'B',
       'Gaz, Électrique': 'GE', 'Bois, Fioul': 'BF', 'Gaz, Bois': 'GB', 'Électrique, Bois': 'EB',
       'Fioul, Bois': 'FB', 'Bois, Électrique': 'BE'}

# Evite les problèmes d'encodage
rentals_df['chauffage_energie'] = rentals_df['chauffage_energie'].replace(chauffage_energie_mapping)

# On se base sur le chauffage mode
rentals_df = rentals_df.drop('chauffage_systeme', axis=1)

# Création de la target
rentals_df['prix_m2_location'] = rentals_df['prix_bien'] / rentals_df['surface']

In [1114]:
rentals_df = rentals_df.dropna(subset=['annee_construction'], axis=0)

In [1116]:
# % de manquants après traitement
round((rentals_df.isna().sum().sum() / sales_df.size) * 100, 2)

0.0

### 3) Gestion des types

In [1118]:
rentals_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11397 entries, immo-facile-52417007 to 115190141
Data columns (total 42 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   type_annonceur           11397 non-null  object 
 1   typedebien               11397 non-null  object 
 2   typedetransaction        11397 non-null  object 
 3   etage                    11397 non-null  int64  
 4   surface                  11397 non-null  int64  
 5   nb_pieces                11397 non-null  int64  
 6   prix_bien                11397 non-null  int64  
 7   mensualiteFinance        11397 non-null  int64  
 8   balcon                   11397 non-null  int64  
 9   eau                      11397 non-null  int64  
 10  bain                     11397 non-null  int64  
 11  dpeL                     11397 non-null  object 
 12  dpeC                     11397 non-null  float64
 13  mapCoordonneesLatitude   11397 non-null  float64
 14  mapC

In [1120]:
rentals_df['nb_etages'] = rentals_df['nb_etages'].astype(int)
rentals_df['nb_toilettes'] = rentals_df['nb_toilettes'].astype(int)
rentals_df['nb_terraces'] = rentals_df['nb_terraces'].astype(int)

rentals_df['videophone'] = np.where(rentals_df['videophone'] == True,  1, 0).astype(int)
rentals_df['porte_digicode'] = np.where(rentals_df['porte_digicode'] == True,  1, 0).astype(int)
rentals_df['cave'] = np.where(rentals_df['cave'] == True,  1, 0).astype(int)
rentals_df['ascenseur'] = np.where(rentals_df['cave'] == True,  1, 0).astype(int)

rentals_df['logement_neuf'] = rentals_df['logement_neuf'].replace({'n': 0, 'o': 1}).astype(int)

rentals_df['annonce_exclusive'] = rentals_df['annonce_exclusive'].replace({'Non': 0, 'Oui': 1, 'O': 1}).astype(int)
rentals_df['date'] = pd.to_datetime(rentals_df['date'], format='%Y-%m')

# Création de colonnes pour les dates
rentals_df['mois'] = rentals_df['date'].dt.month
rentals_df['annee'] = rentals_df['date'].dt.year
 
rentals_df.dtypes

type_annonceur                     object
typedebien                         object
typedetransaction                  object
etage                               int64
surface                             int64
nb_pieces                           int64
prix_bien                           int64
mensualiteFinance                   int64
balcon                              int64
eau                                 int64
bain                                int64
dpeL                               object
dpeC                              float64
mapCoordonneesLatitude            float64
mapCoordonneesLongitude           float64
annonce_exclusive                   int32
nb_etages                           int32
cave                                int32
exposition                         object
ges_class                          object
annee_construction                float64
nb_toilettes                        int32
nb_terraces                         int32
videophone                        

### 4) Encodage

In [1122]:
print("Nombres de variables catégorielles :", len(rentals_df.select_dtypes('object').columns))

Nombres de variables catégorielles : 12


In [1124]:
rentals_df = pd.get_dummies(rentals_df, 
                            columns=['type_annonceur', 'categorie_annonceur', 'typedebien_lite', 'TYP_IRIS_x', 'TYP_IRIS_y', 
                                     'typedebien', 'typedetransaction', 'exposition', 'chauffage_energie'], 
                            drop_first=True)

# Encodage Ordinal
ges_class_mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'NS': 7, 'VI': 8}
dpeL_mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, '0': 7, 'NS': 8, 'VI': 9}
chauffage_mode_mapping = {'Non spécifié': 0, 'Individuel': 1, 'Collectif': 2, 'Central': 3,
                          'Individuel, Central': 4, 'Collectif, Central': 5, 'Collectif, Individuel': 6}

rentals_df['ges_class'] = rentals_df['ges_class'].replace(ges_class_mapping).astype(int)
rentals_df['dpeL'] = rentals_df['dpeL'].replace(dpeL_mapping).astype(int)
rentals_df['chauffage_mode'] = rentals_df['chauffage_mode'].replace(chauffage_mode_mapping).astype(int)

In [1126]:
print("Variables catégorielles restantes :", len(rentals_df.select_dtypes('object').columns))

Variables catégorielles restantes : 0


### 5) Dump

In [1135]:
output_path = '../data/processed/rentals_data_transformed.csv'
rentals_df.to_csv(output_path, index=False)

### 6) Vérification liens entre les datasets ventes et locations

In [30]:
common_ids = pd.merge(sales_df, rentals_df, on="idannonce", how='inner')

display(common_ids.shape)

(0, 109)

<div class="alert alert-info">
<i class="fa fa-info-circle"></i> Nous n'avons aucun lien en commun entre notre dataset ventes 68 et locations 68. Nous ne travaillerons plus avec le dataset locations car il ne va pas répondre à nos objectifs.
</div>

---

### Merge du dataset ventes 68 avec le dataset data-gouv


---

In [12]:
url_sales = 'https://raw.githubusercontent.com/klopstock-dviz/immo_vis/master/data/ech_annonces_ventes_68.csv'
sales_df = pd.read_csv(url_sales, sep=';', index_col="idannonce")

In [14]:
output_path = f"../data/raw/full_2024.csv"
dvf_df = pd.read_csv(output_path, low_memory=False, index_col="id_mutation")

In [16]:
df_formatted = sales_df.copy()
df_formatted = df_formatted.rename(columns={
    "prix_bien": "valeur_fonciere",
    "surface": "surface_reelle_bati",
    "surface_terrain": "surface_terrain",
    "nb_pieces": "nombre_pieces_principales",
    "mapCoordonneesLatitude": "latitude",
    "mapCoordonneesLongitude": "longitude",
    "date": "date_mutation",
    "typedetransaction": "nature_mutation",
    "INSEE_COM": "code_commune"
})
df_formatted.head()

,type_annonceur,typedebien,nature_mutation,etage,surface_reelle_bati,surface_terrain,nombre_pieces_principales,valeur_fonciere,prix_maison,prix_terrain,mensualiteFinance,balcon,eau,bain,dpeL,dpeC,latitude,longitude,annonce_exclusive,nb_etages,parking,places_parking,cave,exposition,ges_class,annee_construction,nb_toilettes,nb_terraces,videophone,porte_digicode,surface_balcon,ascenseur,nb_logements_copro,charges_copro,chauffage_energie,chauffage_systeme,chauffage_mode,categorie_annonceur,logement_neuf,duree_int,typedebien_lite,date_mutation,code_commune,IRIS,CODE_IRIS,TYP_IRIS_x,TYP_IRIS_y,GRD_QUART,UU2010,REG,DEP,loyer_m2_median_n6,nb_log_n6,taux_rendement_n6,loyer_m2_median_n7,nb_log_n7,taux_rendement_n7,prix_m2_vente
idannonce,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
hektor-robischung-2862,pr,a,v,0,64,NaN,3,128400,NaN,NaN,0,1,0,1,C,118.0,47.55424,7.34766,Non,1.0,NaN,NaN,False,0,C,2010.0,1.0,NaN,NaN,NaN,NaN,False,21.0,NaN,Gaz,NaN,Individuel,a,n,NaN,a,2023-04,68325,0,683250000,Z,Z,6832500,68113,44,68,NaN,NaN,NaN,NaN,NaN,NaN,2006.25
apimo-82452515,pr,m,v,0,85,4950.0,4,189000,NaN,NaN,0,0,0,0,0,NaN,48.17073,7.10498,0,NaN,NaN,NaN,NaN,Sud,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Électrique,Radiateur,Individuel,a,n,NaN,m,2023-04,68044,0,680440000,Z,Z,6804400,68000,44,68,NaN,NaN,NaN,NaN,NaN,NaN,2223.53
immo-facile-52471952,pr,m,v,0,187,3182.0,9,282900,NaN,NaN,0,0,0,0,E,251.0,48.16989,7.13037,Oui,NaN,NaN,NaN,NaN,0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,NaN,m,2023-04,68044,0,680440000,Z,Z,6804400,68000,44,68,NaN,NaN,NaN,NaN,NaN,NaN,1512.83
immo-facile-52477215,pr,a,v,0,76,NaN,4,79000,NaN,NaN,0,0,1,0,G,379.0,48.17061,7.11870,Oui,1.0,NaN,NaN,True,0,G,NaN,1.0,1.0,NaN,NaN,NaN,False,2.0,180.0,Fioul,Radiateur,Collectif,a,n,8.0,a,2023-04,68044,0,680440000,Z,Z,6804400,68000,44,68,7.93,3.0,0.092,7.93,3.0,0.092,1039.47
ag681088-383601401,pr,m,v,0,90,3071.0,5,317000,NaN,NaN,0,0,0,1,NS,NaN,47.88536,7.19668,Non,NaN,NaN,2.0,True,0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,NaN,m,2023-04,68159,0,681590000,Z,Z,6815900,68000,44,68,9.77,2.0,0.033,9.77,2.0,0.033,3522.22


In [18]:
df_formatted = df_formatted.drop(['type_annonceur',
                          'parking', 
                          'places_parking',
                          'surface_balcon',
                          'charges_copro',
                          'duree_int',
                          'loyer_m2_median_n6',
                          'nb_log_n6',
                          'taux_rendement_n6',
                          'loyer_m2_median_n7',
                          'nb_log_n7',
                          'taux_rendement_n7'], axis=1)

In [20]:
df_formatted.head()

,typedebien,nature_mutation,etage,surface_reelle_bati,surface_terrain,nombre_pieces_principales,valeur_fonciere,prix_maison,prix_terrain,mensualiteFinance,balcon,eau,bain,dpeL,dpeC,latitude,longitude,annonce_exclusive,nb_etages,cave,exposition,ges_class,annee_construction,nb_toilettes,nb_terraces,videophone,porte_digicode,ascenseur,nb_logements_copro,chauffage_energie,chauffage_systeme,chauffage_mode,categorie_annonceur,logement_neuf,typedebien_lite,date_mutation,code_commune,IRIS,CODE_IRIS,TYP_IRIS_x,TYP_IRIS_y,GRD_QUART,UU2010,REG,DEP,prix_m2_vente
idannonce,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
hektor-robischung-2862,a,v,0,64,NaN,3,128400,NaN,NaN,0,1,0,1,C,118.0,47.55424,7.34766,Non,1.0,False,0,C,2010.0,1.0,NaN,NaN,NaN,False,21.0,Gaz,NaN,Individuel,a,n,a,2023-04,68325,0,683250000,Z,Z,6832500,68113,44,68,2006.25
apimo-82452515,m,v,0,85,4950.0,4,189000,NaN,NaN,0,0,0,0,0,NaN,48.17073,7.10498,0,NaN,NaN,Sud,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Électrique,Radiateur,Individuel,a,n,m,2023-04,68044,0,680440000,Z,Z,6804400,68000,44,68,2223.53
immo-facile-52471952,m,v,0,187,3182.0,9,282900,NaN,NaN,0,0,0,0,E,251.0,48.16989,7.13037,Oui,NaN,NaN,0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,m,2023-04,68044,0,680440000,Z,Z,6804400,68000,44,68,1512.83
immo-facile-52477215,a,v,0,76,NaN,4,79000,NaN,NaN,0,0,1,0,G,379.0,48.17061,7.11870,Oui,1.0,True,0,G,NaN,1.0,1.0,NaN,NaN,False,2.0,Fioul,Radiateur,Collectif,a,n,a,2023-04,68044,0,680440000,Z,Z,6804400,68000,44,68,1039.47
ag681088-383601401,m,v,0,90,3071.0,5,317000,NaN,NaN,0,0,0,1,NS,NaN,47.88536,7.19668,Non,NaN,True,0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,m,2023-04,68159,0,681590000,Z,Z,6815900,68000,44,68,3522.22


In [22]:
# type de bien
df_formatted['typedebien'].unique()
# on perd ici la notion neuf / ancien 
df_formatted = df_formatted.rename(columns={'typedebien':'type_local'})
df_formatted['type_local'] = df_formatted['type_local'].replace({'a': 'Appartement', 
                                                                 'an': 'Appartement', 
                                                                 'm': 'Maison',
                                                                 'mn': 'Maison', 
                                                                 'l': 'Local industriel. commercial ou assimilé'})
# code_type_local sera traité lors du pré-processing

# numero de disposition (regroupement de transactions dans le même acte)
# on a pas cette info dans ventes 68
df_formatted['numero_disposition'] = 1
# nombre_lots correspond au nombre total de lots par disposition
df_formatted['nombre_lots'] = 1

In [24]:
# id_mutation
df_formatted['id_mutation'] = df_formatted['date_mutation'].str[:4] + '-' + df_formatted.index

# on ne garde pas les projets immo 'pi' (17 lignes)
display(df_formatted['nature_mutation'].value_counts())
df_formatted = df_formatted[df_formatted['nature_mutation'].isin(['v', 'vp'])]

df_formatted['nature_mutation'] = np.where(df_formatted['nature_mutation'] == 'v', 'Vente', "Vente en l'état futur d'achèvement")

nature_mutation
v     27369
vp      351
pi       17
Name: count, dtype: int64

In [26]:
# Infos adresses (récupération depuis dataset insee)
url_insee = 'https://raw.githubusercontent.com/klopstock-dviz/immo_vis/master/data/codesPostaux_communesINSEE.csv'
df_insee = pd.read_csv(url_insee, sep=';')
df_insee.head()

,Code_commune_INSEE,Nom_commune,Code_postal,Ligne_5,Libellé_d_acheminement,coordonnees_gps
0,01011,APREMONT,1100,NaN,APREMONT,"46.2054981558,5.65781475272"
1,01021,ARS SUR FORMANS,1480,NaN,ARS SUR FORMANS,"45.9934611,4.82199649494"
2,01028,BANEINS,1990,NaN,BANEINS,"46.1141214445,4.90146475101"
3,01031,BELLIGNAT,1100,NaN,BELLIGNAT,"46.2373497134,5.6360052884"
4,01033,VALSERHONE,1200,BELLEGARDE SUR VALSERINE,VALSERHONE,"46.1067901755,5.83202736464"


In [28]:
# nom_commune, code postal
df_formatted['code_commune'] = df_formatted['code_commune'].astype(str)
df_formatted = df_formatted.merge(
    df_insee[['Code_commune_INSEE', 'Nom_commune', 'Code_postal']].rename(
        columns={'Nom_commune': 'nom_commune', 'Code_postal': 'code_postal'}
    ),
    left_on='code_commune',
    right_on='Code_commune_INSEE',
    how='left'
).set_index('id_mutation')
df_formatted.drop(columns=['Code_commune_INSEE'], inplace=True)

df_formatted['code_postal'] = df_formatted['code_postal'].astype(str).str[:-1]
df_formatted['nom_commune'] = df_formatted['nom_commune'].astype(str).str.capitalize()

#  on a pas les jours au niveau de la date
df_formatted['date_mutation'] = df_formatted['date_mutation'].astype(str) + '-01'

In [30]:
# on créé le code department 68 (n'exite pas dans data-gouv raison géopolitique historique)
df_formatted['code_departement'] = '68'

# creation d'un id_parcelle hypothétique
# rapprochement fait avec le dataset de location qu contient d'autres départements que le 68. 
# ex pour le dep 69
# INSEE_COM 	IRIS 	CODE_IRIS 	TYP_IRIS_x 	TYP_IRIS_y 	GRD_QUART 	UU2010 	REG 	DEP									
# 69383 	    105 	693830105 	H 	        A 	        6938301 	758 	84 	    69
print(dvf_df[dvf_df['code_departement'] == '69']['id_parcelle'].sample(5))

df_formatted['id_parcelle'] = df_formatted[['code_commune', 'TYP_IRIS_y', 'TYP_IRIS_x', 'UU2010']].astype(str).apply(lambda x: ''.join(x), axis=1)

id_mutation
2024-372280    69243000AE0250
2024-376801    690290000B2227
2024-374630    69256000AT0909
2024-366735    69069000AD0093
2024-374374    69199000AH0685
Name: id_parcelle, dtype: object


In [32]:
# pas d'info  dans le dataset ventes 68 
# code_nature_culture, nature_culture
df_formatted['nature_culture'] = 'sols'
df_formatted['code_nature_culture'] = 'S'

# Creation des colonnes manquantes dans df_formatted
missing_cols = [col for col in dvf_df.columns if col not in df_formatted.columns]
for col in missing_cols:
    df_formatted[col] = np.nan

In [34]:
drop_cols = set(df_formatted.columns) - set(dvf_df.columns)
# on garde "prix_m2_vente"
drop_cols.remove('prix_m2_vente')
df_formatted = df_formatted.drop(drop_cols, axis=1)

# suppression anciennes colonnes
drop_cols = [col for col in df_formatted.columns if col not in dvf_df.columns]
# on garde "prix_m2_vente"
drop_cols.remove('prix_m2_vente')
df_formatted = df_formatted.drop(columns=(drop_cols))

In [36]:
df_formatted.head()

,type_local,nature_mutation,surface_reelle_bati,surface_terrain,nombre_pieces_principales,valeur_fonciere,latitude,longitude,date_mutation,code_commune,prix_m2_vente,numero_disposition,nombre_lots,nom_commune,code_postal,code_departement,id_parcelle,nature_culture,code_nature_culture,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,ancien_code_commune,ancien_nom_commune,ancien_id_parcelle,numero_volume,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,code_type_local,code_nature_culture_speciale,nature_culture_speciale
id_mutation,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-hektor-robischung-2862,Appartement,Vente,64,NaN,3,128400,47.55424,7.34766,2023-04-01,68325,2006.25,1,1,Steinsoultz,6864,68,68325ZZ68113,sols,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-apimo-82452515,Maison,Vente,85,4950.0,4,189000,48.17073,7.10498,2023-04-01,68044,2223.53,1,1,Le bonhomme,6865,68,68044ZZ68000,sols,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-immo-facile-52471952,Maison,Vente,187,3182.0,9,282900,48.16989,7.13037,2023-04-01,68044,1512.83,1,1,Le bonhomme,6865,68,68044ZZ68000,sols,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-immo-facile-52477215,Appartement,Vente,76,NaN,4,79000,48.17061,7.11870,2023-04-01,68044,1039.47,1,1,Le bonhomme,6865,68,68044ZZ68000,sols,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-ag681088-383601401,Maison,Vente,90,3071.0,5,317000,47.88536,7.19668,2023-04-01,68159,3522.22,1,1,Jungholtz,6850,68,68159ZZ68000,sols,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
output_path = f"../data/raw/full_years.csv.gz"
dvf_df = pd.read_csv(output_path, low_memory=False, index_col="id_mutation")

In [40]:
df_merged = pd.concat([dvf_df, df_formatted])

In [42]:
output_merge_path = "../data/raw/full_years_merged.csv.gz"
df_merged.to_csv(output_merge_path, compression='gzip')
print(df_merged.shape)

(22145742, 40)


> **Vérification merge**

In [57]:
merged_df = pd.read_csv(output_merge_path, low_memory=False, index_col="id_mutation")
merged_df.head()

,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,ancien_code_commune,ancien_nom_commune,id_parcelle,ancien_id_parcelle,numero_volume,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude,prix_m2_vente
id_mutation,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-1,2019-07-01,1,Vente,244200.0,73.0,NaN,RUE COMMANDANT COUSTEAU,0958,1000.0,01053,Bourg-en-Bresse,01,NaN,NaN,01053000CN0405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,150.0,5.0,S,sols,NaN,NaN,695.0,5.264531,46.194443,NaN
2019-2,2019-07-04,1,Vente,195000.0,852.0,NaN,CHE DE LA CROIX ROSIER,0170,1250.0,01184,Hautecourt-Romanèche,01,NaN,NaN,01184000AM0237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,94.0,4.0,AG,terrains d'agrément,NaN,NaN,888.0,5.415689,46.159272,NaN
2019-2,2019-07-04,1,Vente,195000.0,852.0,NaN,CHE DE LA CROIX ROSIER,0170,1250.0,01184,Hautecourt-Romanèche,01,NaN,NaN,01184000AM0237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,94.0,4.0,S,sols,NaN,NaN,500.0,5.415689,46.159272,NaN
2019-3,2019-07-04,1,Vente,110030.0,5041.0,NaN,LA VILLE,B097,1250.0,01177,Grand-Corent,01,NaN,NaN,011770000B0764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,90.0,3.0,S,sols,NaN,NaN,522.0,5.431065,46.199271,NaN
2019-4,2019-07-03,1,Vente,160450.0,NaN,NaN,CHARBONNET,B039,1270.0,01108,Coligny,01,NaN,NaN,01108000AH0010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,S,sols,NaN,NaN,8.0,5.353051,46.384396,NaN


In [55]:
merged_df.tail()

,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,ancien_code_commune,ancien_nom_commune,id_parcelle,ancien_id_parcelle,numero_volume,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude,prix_m2_vente
id_mutation,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-138910483,2019-02-01,1,Vente,122000.0,NaN,NaN,NaN,NaN,6830.0,68297,St louis,68,NaN,NaN,68297HH68403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,Appartement,26.0,1.0,S,sols,NaN,NaN,NaN,7.56462,47.58427,4692.31
2019-129488827,2019-02-01,1,Vente,202900.0,NaN,NaN,NaN,NaN,6859.0,68285,Rorschwihr,68,NaN,NaN,68285ZZ68000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,Appartement,83.0,4.0,S,sols,NaN,NaN,NaN,7.36406,48.21706,2444.58
2019-129488829,2019-02-01,1,Vente,218400.0,NaN,NaN,NaN,NaN,6859.0,68285,Rorschwihr,68,NaN,NaN,68285ZZ68000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,Appartement,90.0,4.0,S,sols,NaN,NaN,NaN,7.36406,48.21706,2426.67
2019-139483193,2019-02-01,1,Vente,222900.0,NaN,NaN,NaN,NaN,6839.0,68022,Battenheim,68,NaN,NaN,68022ZZ68000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,Appartement,89.0,5.0,S,sols,NaN,NaN,NaN,7.38146,47.81615,2504.49
2019-139483195,2019-02-01,1,Vente,225090.0,NaN,NaN,NaN,NaN,6839.0,68022,Battenheim,68,NaN,NaN,68022ZZ68000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,Appartement,83.0,4.0,S,sols,NaN,NaN,NaN,7.38146,47.81615,2711.93


---

### **Preprocessing du dataset sur les valeurs foncières**

---

In [3]:
output_path = f"../data/raw/full_years.csv.gz"
dvf_df = pd.read_csv(output_path, low_memory=False, index_col="id_mutation")

In [5]:
dvf_df.head()

,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,ancien_code_commune,ancien_nom_commune,id_parcelle,ancien_id_parcelle,numero_volume,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
id_mutation,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-1,2019-07-01,1,Vente,244200.0,73.0,NaN,RUE COMMANDANT COUSTEAU,0958,1000.0,01053,Bourg-en-Bresse,01,NaN,NaN,01053000CN0405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,150.0,5.0,S,sols,NaN,NaN,695.0,5.264531,46.194443
2019-2,2019-07-04,1,Vente,195000.0,852.0,NaN,CHE DE LA CROIX ROSIER,0170,1250.0,01184,Hautecourt-Romanèche,01,NaN,NaN,01184000AM0237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,94.0,4.0,AG,terrains d'agrément,NaN,NaN,888.0,5.415689,46.159272
2019-2,2019-07-04,1,Vente,195000.0,852.0,NaN,CHE DE LA CROIX ROSIER,0170,1250.0,01184,Hautecourt-Romanèche,01,NaN,NaN,01184000AM0237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,94.0,4.0,S,sols,NaN,NaN,500.0,5.415689,46.159272
2019-3,2019-07-04,1,Vente,110030.0,5041.0,NaN,LA VILLE,B097,1250.0,01177,Grand-Corent,01,NaN,NaN,011770000B0764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,90.0,3.0,S,sols,NaN,NaN,522.0,5.431065,46.199271
2019-4,2019-07-03,1,Vente,160450.0,NaN,NaN,CHARBONNET,B039,1270.0,01108,Coligny,01,NaN,NaN,01108000AH0010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,S,sols,NaN,NaN,8.0,5.353051,46.384396


### 1) Suppression colonnes

In [7]:
dvf_df = dvf_df.drop(['ancien_code_commune',
                      'ancien_nom_commune',
                      'ancien_id_parcelle', 
                      'numero_volume',
                      'adresse_suffixe',
                      'adresse_numero',
                      'adresse_nom_voie',
                      'adresse_code_voie'], axis=1)

In [29]:
dvf_df = dvf_df[~dvf_df.index.duplicated(keep='first')]

### 2) Gestion des manquants

In [9]:
# % de manquants avant traitement
round((dvf_df.isna().sum().sum() / dvf_df.size) * 100, 2)

46.06

In [11]:
dvf_df.loc[(dvf_df['code_type_local'].isna()) & (dvf_df['type_local'] == 'Maison'), 'code_type_local'] = 1.0
dvf_df.loc[(dvf_df['code_type_local'].isna()) & (dvf_df['type_local'] == 'Appartement'), 'code_type_local'] = 2.0
dvf_df.loc[(dvf_df['code_type_local'].isna()) & (dvf_df['type_local'] == 'Dépendance'), 'code_type_local'] = 3.0
dvf_df.loc[(dvf_df['code_type_local'].isna()) & (dvf_df['type_local'] == 'Local industriel. commercial ou assimilé'), 'code_type_local'] = 4.0
dvf_df.loc[(dvf_df['code_type_local'].isna()), 'code_type_local'] = 5.0

dvf_df.loc[(dvf_df['type_local'].isna()) & (dvf_df['code_type_local'] == 1.0), 'type_local'] = 'Maison'
dvf_df.loc[(dvf_df['type_local'].isna()) & (dvf_df['code_type_local'] == 2.0), 'type_local'] = 'Appartement'
dvf_df.loc[(dvf_df['type_local'].isna()) & (dvf_df['code_type_local'] == 3.0), 'type_local'] = 'Dépendance'
dvf_df.loc[(dvf_df['type_local'].isna()) & (dvf_df['code_type_local'] == 4.0), 'type_local'] = 'Local industriel. commercial ou assimilé'
dvf_df.loc[(dvf_df['type_local'].isna()), 'type_local'] = 'Autre'

dvf_df.loc[(dvf_df['surface_reelle_bati'].isna()) & (dvf_df['code_nature_culture'] != "AB"), 'surface_reelle_bati'] = 0

dvf_df.loc[(dvf_df['nombre_pieces_principales'].isna()) & (dvf_df['code_nature_culture'] != "AB"), 'nombre_pieces_principales'] = 0

dvf_df['lot1_surface_carrez'] = dvf_df['surface_reelle_bati']

dvf_df['code_nature_culture'] = dvf_df['code_nature_culture'].fillna('NS')

dvf_df['nature_culture'] = dvf_df['nature_culture'].fillna('autres')

dvf_df['code_nature_culture_speciale'] = dvf_df['code_nature_culture_speciale'].fillna('NS')

dvf_df['nature_culture_speciale'] = dvf_df['nature_culture_speciale'].fillna('Autre')

dvf_df['surface_terrain'] = dvf_df['surface_terrain'].fillna(0)

dvf_df = dvf_df.dropna(subset=['longitude', 'latitude', 'valeur_fonciere'], axis=0)

dvf_df['surface_reelle_bati'] = dvf_df['surface_reelle_bati'].fillna(0)

dvf_df['lot1_surface_carrez'] = dvf_df['lot1_surface_carrez'].fillna(0)
dvf_df['lot2_surface_carrez'] = dvf_df['lot2_surface_carrez'].fillna(0)
dvf_df['lot3_surface_carrez'] = dvf_df['lot3_surface_carrez'].fillna(0)
dvf_df['lot4_surface_carrez'] = dvf_df['lot4_surface_carrez'].fillna(0)
dvf_df['lot5_surface_carrez'] = dvf_df['lot5_surface_carrez'].fillna(0)

dvf_df['lot1_numero'] = dvf_df['lot1_numero'].fillna(0)
dvf_df['lot2_numero'] = dvf_df['lot2_numero'].fillna(0)
dvf_df['lot3_numero'] = dvf_df['lot3_numero'].fillna(0)
dvf_df['lot4_numero'] = dvf_df['lot4_numero'].fillna(0)
dvf_df['lot5_numero'] = dvf_df['lot5_numero'].fillna(0)

dvf_df['nombre_pieces_principales'] = dvf_df['nombre_pieces_principales'].fillna(0)

In [13]:
# on garde le code commune (0 manquant et réduit la multicolinéarité)
dvf_df = dvf_df.drop('code_postal', axis=1)

In [15]:
# % de manquants après traitement
round((dvf_df.isna().sum().sum() / dvf_df.size) * 100, 2)

0.0

### 3) Gestion des types

In [17]:
dvf_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21421873 entries, 2019-1 to 2024-1218061
Data columns (total 30 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   date_mutation                 object 
 1   numero_disposition            int64  
 2   nature_mutation               object 
 3   valeur_fonciere               float64
 4   code_commune                  object 
 5   nom_commune                   object 
 6   code_departement              object 
 7   id_parcelle                   object 
 8   lot1_numero                   object 
 9   lot1_surface_carrez           float64
 10  lot2_numero                   object 
 11  lot2_surface_carrez           float64
 12  lot3_numero                   object 
 13  lot3_surface_carrez           float64
 14  lot4_numero                   object 
 15  lot4_surface_carrez           float64
 16  lot5_numero                   object 
 17  lot5_surface_carrez           float64
 18  nombre_lots     

In [19]:
dvf_df = dvf_df.copy()

dvf_df['lot1_numero'] = pd.to_numeric(dvf_df['lot1_numero'], errors='coerce').fillna(0).astype(int)
dvf_df['lot2_numero'] = pd.to_numeric(dvf_df['lot2_numero'], errors='coerce').fillna(0).astype(int)
dvf_df['lot3_numero'] = pd.to_numeric(dvf_df['lot3_numero'], errors='coerce').fillna(0).astype(int)
dvf_df['lot4_numero'] = pd.to_numeric(dvf_df['lot4_numero'], errors='coerce').fillna(0).astype(int)
dvf_df['lot5_numero'] = pd.to_numeric(dvf_df['lot5_numero'], errors='coerce').fillna(0).astype(int)

dvf_df = dvf_df[dvf_df['date_mutation'] != 0]
dvf_df['date_mutation'] = pd.to_datetime(dvf_df['date_mutation'], format='%Y-%m-%d')

dvf_df['annee'] = dvf_df['date_mutation'].dt.year
dvf_df['mois'] = dvf_df['date_mutation'].dt.month
dvf_df['jour'] = dvf_df['date_mutation'].dt.day

dvf_df.dtypes

date_mutation                   datetime64[ns]
numero_disposition                       int64
nature_mutation                         object
valeur_fonciere                        float64
code_commune                            object
nom_commune                             object
code_departement                        object
id_parcelle                             object
lot1_numero                              int32
lot1_surface_carrez                    float64
lot2_numero                              int32
lot2_surface_carrez                    float64
lot3_numero                              int32
lot3_surface_carrez                    float64
lot4_numero                              int32
lot4_surface_carrez                    float64
lot5_numero                              int32
lot5_surface_carrez                    float64
nombre_lots                              int64
code_type_local                        float64
type_local                              object
surface_reell

### 4) Encodage

In [21]:
print("Nombres de variables catégorielles :", len(dvf_df.select_dtypes('object').columns))

Nombres de variables catégorielles : 10


In [35]:
# on a le code commune
dvf_df = dvf_df.drop('nom_commune', axis=1)

dvf_df['nature_mutation'] = np.where(dvf_df['nature_mutation'] == 'Vente', 1, 0)

# Cas spécifique pour la Corse
# code departement
dvf_df['code_departement'] = dvf_df['code_departement'].replace({'2A': 201, '2B': 202}).astype(int)
# code_commune
dvf_df.loc[dvf_df['code_commune'].str.startswith('2A'), 'code_commune'] = dvf_df['code_commune'].str.replace('2A', '201')
dvf_df.loc[dvf_df['code_commune'].str.startswith('2B'), 'code_commune'] = dvf_df['code_commune'].str.replace('2B', '202')

dvf_df = dvf_df.drop('id_parcelle', axis=1)
# on a le code correspondant
dvf_df = dvf_df.drop(['nature_culture_speciale', 'nature_culture'], axis=1)

dvf_df['code_commune'] = dvf_df['code_commune'].astype(int)
dvf_df['hashed_code_commune'] = dvf_df['code_commune'].astype(int)

In [39]:
# Frequency Encoding
freq_cols = ['type_local', 'code_nature_culture', 'code_nature_culture_speciale']
for col in freq_cols:
    freq_encoding = dvf_df[col].value_counts(normalize=True)
    dvf_df[col] = dvf_df[col].map(freq_encoding)

In [41]:
print("Variables catégorielles restantes :", len(dvf_df.select_dtypes('object').columns))

Variables catégorielles restantes : 0


### 5) Dump

In [49]:
# Pour suivre l'évolution temporelle
output_path = '../data/processed/dvf_data_transformed.csv'
dvf_df.to_csv(output_path, index=False)

# Pour la prédiction de la valeur foncière (seulement la dernière année)
output_path_2024 = '../data/processed/dvf_data_2024_transformed.csv'

dvf_data_2024 = dvf_df[dvf_df['annee'] == 2024]
dvf_data_2024.to_csv(output_path_2024, index=False)